# Docking with Gnina

In this quickstart guide we will see how to dock a small molecule into a protein using the Gnina module. 

# Install

First, install the following modules via the command-line (we require Python ≥ 3.9):

In [2]:
pip install rush-py pdb-tools py3Dmol

Note: you may need to restart the kernel to use updated packages.


# Full Code

In [4]:
from pdbtools import (
    pdb_delhetatm,
    pdb_fetch,
    pdb_selchain,
)
import py3Dmol
import rush

# create the rush client
client = rush.build_blocking_provider_with_functions(access_token = "PUT_YOUR_TOKEN_HERE")

# download the PDB structure using its RCSB ID
structure = list(pdb_fetch.fetch_structure("1B39"))

# select the protein chain and save it to a file
protein = pdb_delhetatm.remove_hetatm(pdb_selchain.select_chain(structure, "A"))
with open(client.workspace / "protein.pdb", "w") as f:
    for line in protein:
        f.write(str(line))

# run protein preparation which will use a combination of
# PDBFixer and PDB2PQR to prepare the protein
_, prepared_protein_pdb_handle = client.prepare_protein(
    client.workspace / "protein.pdb", None, None
)

# setup an SMI file that contains the SMILES string of our ligand
ligand_smi_filename = client.workspace / "ligand.smi"
ligand_smi_filename.write_text(
    "c1nc(c2c(n1)n(cn2)[C@H]3[C@@H]([C@@H]([C@H](O3)CO[P@@](=O)(O)O[P@](=O)(O)OP(=O)(O)O)O)O)N 1"
)

# run Auto3D which will give us 3 conformers of our ligand
# in the SDF format and the QDXF format
ligand_sdf_handle, _ = client.auto3d(
    ligand_smi_filename, # the filename that stores our ligand
    "smi",               # the format of the file
    { "k": 3 },          # the number of conformers to generate
    resources={
        "gpus": 1,            # the number of GPUs to use
        "storage": 5,         # the amount of storage to use
        "storage_units": "MB" # the units of storage (here we are using megabytes)
    }
)

# run Auto3D which will give us 3 conformers of our ligand
# in the SDF format and the QDXF format
poses_sdf_handle, scores = client.gnina_pdb(
    protein_pdb_handle,
    ligand_sdf_handle,
    "sdf",
    # resources
    resources={
        "gpus": 1,            # the number of GPUs to use
        "storage": 10,        # the amount of storage to use
        "storage_units": "MB" # the units of storage (here we are using megabytes)
    },
)

# print the status of all jobs
print(client.status())

# download the results (this will block until the Gnina job has completed)
protein_pdb = protein_pdb_handle.download()
poses_sdf = poses_sdf_handle.download()

# visualize the results in 3D
view = py3Dmol.view()
view.addModel(protein_pdb.read(), "pdb")
view.addModel(poses_sdf.read(), "sdf")
view.setStyle({"cartoon": {"color": "spectrum"}})
view.zoomTo()
view.show()

ValueError: too many values to unpack (expected 1)